In this file I want to take the cleaned CRA dataset and create probabilities of default. Later I will calculate other things such as expected loss and a risk forecast and sensitivity to spikes like recession or pandemic. 

In [3]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data = 'CRA_cleaned_loan_data.csv'
df = pd.read_csv(data)

In [7]:
print(df.head)

<bound method NDFrame.head of        person_age  person_gender  person_education  person_income  \
0            22.0              0               4.0        71948.0   
1            21.0              0               1.0        12282.0   
2            25.0              0               1.0        12438.0   
3            23.0              0               3.0        79753.0   
4            24.0              1               4.0        66135.0   
...           ...            ...               ...            ...   
44995        27.0              1               2.0        47971.0   
44996        37.0              0               2.0        65800.0   
44997        33.0              1               2.0        56942.0   
44998        29.0              1               3.0        33164.0   
44999        24.0              1               1.0        51609.0   

       person_emp_exp  person_home_ownership  loan_amnt  loan_int_rate  \
0                   0                    0.0    35000.0          16

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


#Specify your features and target
feature_cols = [
    "loan_amount",
    "interest_rate",
    "term_months",
    "dti_ratio",
    "fico_score",
    # … add or remove features as needed
]
X = df.drop('loan_status', axis=1)
y = df["loan_status"]

X = X.dropna()
y = y.loc[X.index]  # Keep target aligned

# 3. Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. (Optional but recommended) Scale your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 5. Fit a logistic‐regression model
model = LogisticRegression(
    penalty="l2",
    C=1.0,
    class_weight="balanced",  # adjust if defaults are rare
    solver="lbfgs",
    max_iter=1000,
)
model.fit(X_train_scaled, y_train)

# 6. Calculate PD = P(default) for each observation in the test set
#    predict_proba returns [[P(non‐default), P(default)], …]
pd_test = model.predict_proba(X_test_scaled)[:, 1]

# 7. Attach PD back to your test frame for analysis
results = X_test.copy()
results["actual_default"] = y_test.values
results["PD"] = pd_test

# 8. Peek at the results
print(results.head(10))


       person_age  person_gender  person_education  person_income  \
15082        25.0              1               1.0       144870.0   
43209        25.0              0               1.0        42734.0   
12808        26.0              0               3.0        88424.0   
36073        23.0              0               2.0        41770.0   
19660        33.0              0               3.0        42930.0   
25531        29.0              1               2.0       103070.0   
41106        30.0              1               2.0        82447.0   
32345        58.0              1               3.0        62551.0   
37404        25.0              0               1.0        60878.0   
17026        26.0              0               1.0        97399.0   

       person_emp_exp  person_home_ownership  loan_amnt  loan_int_rate  \
15082               2                    2.0     3000.0          10.14   
43209               4                    0.0    19000.0          11.63   
12808             